# Project 2: M. tuberculosis Genome Assembly
## 06 - Pangenome Analysis with Roary

* **Author:** Youssef Mimoune
* **Date:** 26-Oct-2025

### Objective
Our hypothesis-driven search (`grep`) for known resistance mutations (`folC`, `thyA`, `ribD`) failed. This suggests our resistant strain (`DRR749572`) has a novel or less common mutation.

We will now perform an **unbiased, pangenome analysis** using `Roary`.

### Methodology
1.  **Run `Roary`:** We will feed `Roary` the `.gff` annotation files from `Prokka`.
2.  **How it works:** `Roary` is "smart". It identifies all genes present in both samples (the "core genome"), regardless of their order in the assembly.
3.  **Output:** `Roary` will create a single alignment file (`core_gene_alignment.aln`) containing all ~4000 core genes, perfectly aligned between the two samples.
4.  **Next Step:** We will then use a tool called `snp-sites` on this alignment to find the exact mutation.

### 2. Running Roary (Manual Step)

Due to a complex software dependency conflict (Prokka's BioPerl vs. Roary's older BioPerl), `Roary` could not be installed in our main `(assembly_env)`.

Therefore, the `Roary` command was run **manually in the terminal** inside a separate, temporary environment (`roary_env`). This "Safe Mode" (`-p 1`) run was slow but successful.

**Command used in terminal:**
```bash
# 1. Clean up old attempts
rm -rf analysis/07_roary_results*

# 2. Activate roary_env
mamba activate roary_env

# 3. Run Roary in "Safe Mode" (-p 1)
roary \
  -p 1 \
  -f analysis/07_roary_results_FINAL \
  -e -n \
  -v \
  analysis/05_prokka_annotation/*/*.gff

# 4. Deactivate and return
mamba deactivate

In [23]:
print("--- 3. Verifying the FINAL Roary Output ---")
# We add ../ to go up from 'notebooks/'
# We look *only* for the '..._FINAL' directory
!ls -lh ../analysis/07_roary_results_FINAL

--- 3. Verifying the FINAL Roary Output ---
total 14M
-rw-rw-r-- 1 refm_youssef refm_youssef 4.2K Oct 26 02:05 accessory.header.embl
-rw-rw-r-- 1 refm_youssef refm_youssef 3.2K Oct 26 02:05 accessory.tab
-rw-rw-r-- 1 refm_youssef refm_youssef    0 Oct 26 02:05 accessory_binary_genes.fa
-rw-rw-r-- 1 refm_youssef refm_youssef  866 Oct 26 02:05 accessory_graph.dot
-rw-rw-r-- 1 refm_youssef refm_youssef   18 Oct 26 02:05 blast_identity_frequency.Rtab
-rw-rw-r-- 1 refm_youssef refm_youssef 154K Oct 26 02:05 clustered_proteins
-rw-rw-r-- 1 refm_youssef refm_youssef 517K Oct 26 02:05 core_accessory.header.embl
-rw-rw-r-- 1 refm_youssef refm_youssef 614K Oct 26 02:05 core_accessory.tab
-rw-rw-r-- 1 refm_youssef refm_youssef 219K Oct 26 02:05 core_accessory_graph.dot
-rw-rw-r-- 1 refm_youssef refm_youssef 444K Oct 26 02:28 core_alignment_header.embl
-rw-rw-r-- 1 refm_youssef refm_youssef 7.5M Oct 26 02:28 core_gene_alignment.aln
-rw-rw-r-- 1 refm_youssef refm_youssef  53K Oct 26 02:05 gene_pres

In [24]:
print("--- 4. Reading the Roary Summary ---")
# We read the summary file from the FINAL directory
!cat ../analysis/07_roary_results_FINAL/summary_statistics.txt

--- 4. Reading the Roary Summary ---
Core genes	(99% <= strains <= 100%)	3967
Soft core genes	(95% <= strains < 99%)	0
Shell genes	(15% <= strains < 95%)	32
Cloud genes	(0% <= strains < 15%)	0
Total genes	(0% <= strains <= 100%)	3999


### 3. Finding Mutations with snp-sites

Now that we have the 7.5MB `core_gene_alignment.aln` file (which contains all ~4000 core genes aligned), we can use `snp-sites` to scan this file and find every single difference (SNP) between our two samples.

In [25]:
print("--- 5. Installing snp-sites (needed to find mutations) ---")
!mamba install -c bioconda snp-sites -y

--- 5. Installing snp-sites (needed to find mutations) ---

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.4.2) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['snp-sites']

[+] 0.0s
bioconda/linux-64

In [27]:
print("--- 6. Running snp-sites (Corrected) to find MUTATIONs ---")
# This is the corrected command.
# We MUST add the '-v' flag to force VCF output (which includes positions)
# -o : output file
# Input: The 7.5MB alignment file

!snp-sites -v -o ../analysis/snps.vcf ../analysis/07_roary_results_FINAL/core_gene_alignment.aln

print("\n--- Differences found! Reading the VCF file (This time it's correct): ---")
# Now we print the contents of the VCF file
!cat ../analysis/snps.vcf

--- 6. Running snp-sites (Corrected) to find MUTATIONs ---

--- Differences found! Reading the VCF file (This time it's correct): ---
##fileformat=VCFv4.1
##contig=<ID=1,length=3851508>
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	DRR749571_control	DRR749572_resistant
1	52975	.	A	G	.	.	.	GT	0	1
1	334765	.	T	C	.	.	.	GT	0	1
1	632914	.	G	C	.	.	.	GT	0	1
1	632916	.	C	G	.	.	.	GT	0	1
1	632917	.	G	A	.	.	.	GT	0	1
1	633293	.	G	C	.	.	.	GT	0	1
1	1320540	.	C	G	.	.	.	GT	0	1
1	1320640	.	G	C	.	.	.	GT	0	1
1	2144007	.	C	T	.	.	.	GT	0	1
1	2338162	.	T	C	.	.	.	GT	0	1
1	2965370	.	G	A	.	.	.	GT	0	1


### 4. Final Conclusion

This is the most important finding of the project.

The `snp-sites` tool (Cell 7) successfully ran and produced a VCF file. This file shows that there are **exactly 11 Single Nucleotide Polymorphisms (SNPs)** that differentiate the core genome of the resistant strain from the control strain.

Furthermore, our investigation in Notebook 05 (the `grep` analysis) proved that the resistance was **NOT** caused by mutations in any of the known canonical target genes (`folP1`, `folP2`, `folC`, `thyA`, or `ribD`), as their protein sequences were all identical.

**Scientific Conclusion:**
The PAS resistance in sample `DRR749572` is caused by a **non-canonical (novel or uncommon) mechanism**. The causative mutation is **one of these 11 SNPs** found in the core genome, which are located in genes not previously associated with PAS resistance.

Further research would involve analyzing these 11 genes to identify which one is the true cause.